# **Tools Recognition**

In [1]:
import os
import cv2
import json
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import scipy.ndimage as nd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

---
### 1) Raccolta e etichettatura delle immagini

Creiamo il nostro dataset di train e test etichettando le varie immagini e considerando varie situazioni: diverse posizioni degli oggetti, cambi di luminosità, presenza di ombre e diversi sfondi, alcuni dei quali presentano delle irregolarità.

- 10 diverse categorie:
    - accendino
    - cacciavite
    - chiave
    - forbici
    - martello
    - metro
    - nastro
    - pappagallo
    - penna
    - spillatrice 

- 4 diverse superfici:
    - tavolo bianco
    - banco da lavoro
    - pavimento di cemento
    - coperta a righe
- 20 immagini per il training per ogni categoria
- 26 immagini per il test contenenti oggetti multipli e anche estranei alle 10 categorie


In [ ]:
# Oggetti utilizzati.
tool_names = ['accendino', 'cacciavite', 'chiave', 'forbici', 'martello', 'metro', 'nastro', 'pappagallo', 'penna', 'spillatrice']

In [ ]:
# Rinomiamo le immagini nelle cartelle.
cwd = os.getcwd()
for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        new_name = tool+ '_' + str(i+1) + '.png'
        os.rename(file, os.path.join(path, new_name))


# Rinominiamo le immagini del test set.
path = os.path.join(cwd, f"images/Test_set/")
image_files = glob.glob(path + '*.png')

for i, file in enumerate(image_files):
    new_name = 'test_' + str(i+1) + '.png'
    os.rename(file, os.path.join(path, new_name))

---
### 2) Pre-elaborazione delle immagini

Prima della segmentazione creiamo un *ground truth* per ogni immagine in modo tale da poter valutare lo step successivo in temini di accuratezza e modifichiamo le immagini per preparale per i prossimi step: ridimensioniamo le immagini riducendole tutte alla stessa dimensione (520 x 520 pixel) e le ricoloriamo in scala di grigi per ridurle ad un singolo canale e renderle più pratiche e semplici da elaborare.

- Ridimensionamento : portartiamo nello stesso formato tutte le immagini (520 x 520). Alcune delle utilità di questo passaggio possono essere una maggiore uniformità una volta che si va ad allenare l'algoritmo di calssificazione e l'aumento della diversità dei dati di training che possono portare il modello ad avere una maggiore robustezza;

In [ ]:
# Portiamo tutte le immagini nella stessa dimensione (520 x 520).
cwd = os.getcwd()

for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        image = Image.open(file)

        # Ridimensionamento dell'immagine.
        resized_image = image.resize((520, 520))
        resized_image.save(file)

# test-set
path = os.path.join(cwd, f"images/Test_set/")
image_files = glob.glob(path + '*.png')

for i, file in enumerate(image_files):
    image = Image.open(file)

    # Ridimensionamento dell'immagine.
    resized_image = image.resize((520, 520))
    resized_image.save(file)

- Ricolorazione: applichiamo la scalda di grigi riducendo le immagini ad un unico canale rendendole più semplici da analizzare;
- Gaussian Filter: applichiamo un `Gaussian filter` con dimensione 5x5 per sfocare l'immagine e rendere lo sfondo piu semplice da distinguere rispetto agli oggetti;

---
### 3) Segmentazione delle immagini 

Una volta ottenuto l'immagine preparata, si applica un metodo di segmentazione per dividere l'immagine in regioni omogenee. Abbiamo provato ad utilizzare tecniche di segmentazione come la segmentazione basata sulla soglia, la segmentazione basata sulla regione o la segmentazione basata sui cluster: queste tecniche non hanno portato buoni risultati perchè, anche se identificano molto bene gli oggetti con una precisione quasi sempre superiore al 90%, con altrettanta precisione sbagliano ad identificare lo sfondo a causa della presenza di ombre e di sfondi che non permetono una chiara distinzione con l'oggetto considerato e presentano delle imperfezioni. Confrontiamo perciò alcune tecniche di segmentazione sulle immagini a sfondo bianco.

Tecniche di segmentazione:
- per contorni: Canny edge detector

- per regioni: sogliatura automatica OTSU

- mediante clustering: k-mean


In [ ]:
tools = ['accendino', 'cacciavite', 'chiave', 'forbici', 'martello', 'metro', 'nastro', 'pappagallo', 'penna', 'spillatrice']
tp_tot, tn_tot, fp_tot, fn_tot = [], [], [], []

cwd = os.getcwd()
folder_path = os.path.join(cwd, f"images")


for tool in tqdm(tools):

    with open(f"{folder_path}\{tool}\{tool}_gt.json", 'r') as f:
        ground_truth_file = json.load(f)

    for pos, el in enumerate(ground_truth_file):

        if pos < 5:
            truth_mask = np.zeros((520, 520))
            image = cv2.imread(f"{folder_path}\{tool}\{tool}_{pos+1}.png")

            for i in range(len(ground_truth_file[el]['regions'])):

                x_pos = ground_truth_file[el]['regions'][i]['shape_attributes']['all_points_x']
                y_pos = ground_truth_file[el]['regions'][i]['shape_attributes']['all_points_y']
                vertices = np.array([[x_pos[u], y_pos[u]] for u in range(len(x_pos))])
                if i == 0:
                        cv2.fillPoly(truth_mask, [vertices], 255)
                else:
                        cv2.fillPoly(truth_mask, [vertices], 0)

            # Ground truth.
            x_pos = ground_truth_file[el]['regions'][0]['shape_attributes']['all_points_x']
            y_pos = ground_truth_file[el]['regions'][0]['shape_attributes']['all_points_y']
            vertices = np.array([[x_pos[i], y_pos[i]] for i in range(len(x_pos))])

            # Preprocessing.
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), cv2.BORDER_DEFAULT)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
            gray = cv2.erode(gray, kernel, iterations=3)


            ## Tecniche di segmentazione.

            # 1) Sogliaura di OTSU
            _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

            # 2) K-means
            small = cv2.pyrDown(gray)
            Z = small.reshape((-1, 1))
            Z = np.float32(Z)
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
            K = 2
            ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
            center = np.uint8(center)
            res = center[label.flatten()]
            res2 = res.reshape((small.shape))
            res2 = cv2.pyrUp(res2)
            _, mask = cv2.threshold(res2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # 3) CANNY Edge Detection
            edges = cv2.Canny(gray, threshold1=100, threshold2=200)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
            closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
            mask = nd.binary_fill_holes(closed_edges)
            mask = mask.astype(np.uint8) * 255


            # cv2.imshow("Masked Image", mask)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()


            # Ground truth mask.
            truth_mask = truth_mask.astype(bool).flatten()
            opening = mask.astype(bool)

            # Confronto tra segmentazione e ground truth.
            ground_truth = truth_mask.flatten()
            predictions = [elem for elem in opening.flatten()]

            tn, fp, fn, tp = confusion_matrix(ground_truth, predictions, labels=[True, False]).ravel()
            tp_tot.append(tp)
            tn_tot.append(tn)
            fp_tot.append(fp)
            fn_tot.append(fn)


# Matrice di Confusione.
sum_tp, sum_tn, sum_fp, sum_fn = sum(tp_tot), sum(tn_tot), sum(fp_tot), sum(fn_tot) 

cf_matrix = np.array([[sum_tp/(sum_tp+sum_fp), sum_fp/(sum_tp+sum_fp)],
                       [sum_fn/(sum_tn+sum_fn), sum_tn/(sum_tn+sum_fn)]])

100%|██████████| 9/9 [00:07<00:00,  1.28it/s]


In [ ]:
labels = ['Tool', 'Background']
sns.heatmap(cf_matrix, annot=True, cmap='viridis',  xticklabels=labels, yticklabels=labels, cbar=False, fmt=".1%")
plt.title("Title")
plt.xlabel('Actual')
plt.ylabel('Predicted');

Problemi:
- **Ombre**, andando a segmentare l'immagine le ombre tendono ad essere classificate come l'oggetto e infatti notiamo come per tutti i metodi ci sia una buona percentuale di falsi positivi, ovvero lo sfondo viene riconosciuto erroneamente come oggetto;
- **Sfondo**, gli sfondi utilizzati creano diversi problemi alle tecinche di segmentazione che non riescono adeliminare i pattern presenti, es. i pallini del tavolo da lavoro o le strisce della coperta, e portano perciò ad una segmentazione degli oggetti erronea, aumetanto notevolmente la percentuale di pixel classificati come oggetti quando questi appartengono allo sfondo.


''' Confrontiamo alcune tecninche di segmentazione sulle immagini a sfondo bianco (vedi Prove Tecninche di Segmentazione) e notiamo come la sogliatura per contorni risulti la migliore perchè riesce ad andare ad identificare in aniera abbastanza corretta i bordi delle immagini non classificando lo sfondo come oggetto. Continuiamo la nostra analisi utilizzando il ground truth per l'estazione delle features e la classificazinoe degli oggetti.'''

### 4) Estrazione delle features

Le features che andiamo ad estrarre da ogni immagine per allenare in nostro modello sono:

- ...
- ...
- ...



### 5) Addestramento del modello di classificazione 

Come modello di classificazione utilizziamo ... 

### 6) Classificazione di oggetti

### PIPELINE

1) ``Raccolta dei dati``: Creiamo il nostro dataset di train e test etichettando le varie immagini e considerando varie situazioni: diverse posizioni degli oggetti, cambi di luminosità, presenza di ombre e diversi sfondi, alcuni dei quali presentano delle irregolarità.

2) ``Pre-elaborazione dell'immagine``: Prima della segmentazione creiamo un *ground truth* per ogni immagine in modo tale da poter valutare lo step successivo in temini di accuratezza e modifichiamo le immagini per preparale per i prossimi step: ridimensioniamo le immagini riducendole tutte alla stessa dimensione (520 x 520 pixel) e le ricoloriamo in scala di grigi per ridurle ad un singolo canale e renderle più pratiche e semplici da elaborare.

3) ``Segmentazione dell'immagine``: Una volta ottenuto l'immagine preparata, si applica un metodo di segmentazione per dividere l'immagine in regioni omogenee. Abbiamo provato ad utilizzare tecniche di segmentazione come la segmentazione basata sulla soglia, la segmentazione basata sulla regione o la segmentazione basata sui cluster: queste tecniche non hanno portato buoni risultati perchè, anche se identificano molto bene gli oggetti con una precisione quasi sempre superiore al 90%, con altrettanta precisione sbagliano ad identificare lo sfondo a causa della presenza di ombre e di sfondi che non permetono una chiara distinzione con l'oggetto considerato e presentano delle imperfezioni. Cerchiamo quindi un'altra tecnica...

---
 


4) ``Estrazione delle features``: Per ogni regione dell'immagine segmentata, si estraggono le feature utilizzate per addestrare il modello di classificazione. Le feature possono includere la forma, la texture, il colore o qualsiasi altra caratteristica che aiuti a distinguere le diverse classi di oggetti.

5) ``Addestramento del modello di classificazione``: Una volta estratte le feature, si addestra il modello di classificazione utilizzando un algoritmo di machine learning come le reti neurali, le SVM o gli alberi di decisione. Il modello viene addestrato per riconoscere le diverse classi di oggetti.

6) ``Rilevamento degli oggetti``: Una volta addestrato il modello di classificazione, si può applicare il modello all'immagine originale per rilevare gli oggetti di interesse. Utilizzando le feature estratte, il modello cerca le regioni dell'immagine che corrispondono agli oggetti di interesse.